In [20]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import pickle

In [21]:
file_path = '../Data/activePlayerCareerLogData.pickle'
PLAYER = 'kyrie irving'
target = 'PTS'
cols_to_drop = ['SEASON_ID', 'Player_ID', 'GAME_DATE', 'OPP', 'TEAM', target + '_target']

In [22]:
# Load collected player data from pickle file
with open(file_path, 'rb') as file:
    activePlayerCareerLog = pickle.load(file)

In [23]:
activePlayerCareerLog['aaron gordon'].head()

,SEASON_ID,Player_ID,GAME_DATE,WL,MIN,FGM,FGA,FG3M,FG3A,FTM,...,HOME,OPP,TEAM,REST,PTS_target,LAST_3_PTS,LAST_5_PTS,LAST_7_PTS,LAST_3_PLUS_MINUS,LAST_5_FGM_PCT
0,22016,203932,2016-10-26,0,35,5,12,2,3,0,...,1,MIA,ORL,4,17.0,NaN,NaN,NaN,NaN,NaN
1,22016,203932,2016-10-28,0,29,6,12,1,2,4,...,0,DET,ORL,2,8.0,NaN,NaN,NaN,NaN,NaN
2,22016,203932,2016-10-29,0,26,3,7,1,3,1,...,0,CLE,ORL,1,7.0,12.333333,NaN,NaN,-14.000000,NaN
3,22016,203932,2016-11-01,1,32,3,10,0,4,1,...,0,PHI,ORL,3,13.0,10.666667,NaN,NaN,-6.666667,NaN
4,22016,203932,2016-11-03,1,31,5,10,1,3,2,...,1,SAC,ORL,2,9.0,9.333333,11.4,NaN,-2.666667,0.429048


In [24]:
for name, player_df in activePlayerCareerLog.items():
    activePlayerCareerLog[name] = player_df.dropna()

In [25]:
df_columns = list(activePlayerCareerLog[PLAYER].columns)
features = df_columns
for col in cols_to_drop:
    features.remove(col)
print(features)

['WL', 'MIN', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'HOME', 'REST', 'LAST_3_PTS', 'LAST_5_PTS', 'LAST_7_PTS', 'LAST_3_PLUS_MINUS', 'LAST_5_FGM_PCT']


In [26]:
# Split players into train and test sets
split_train_ratio = 0.70
split_val_ratio = 0.90
players_train = list(activePlayerCareerLog.keys())[:int(split_train_ratio * len(activePlayerCareerLog))]
players_val = list(activePlayerCareerLog.keys())[int(split_train_ratio * len(activePlayerCareerLog)) : int(split_val_ratio * len(activePlayerCareerLog))]
players_test = list(activePlayerCareerLog.keys())[int(split_val_ratio * len(activePlayerCareerLog)):]
print(len(players_train), len(players_val), len(players_test))

309 88 45


In [27]:
df_train = pd.concat(activePlayerCareerLog[name] for name in players_train)
df_val = pd.concat(activePlayerCareerLog[name] for name in players_val)
df_test = pd.concat(activePlayerCareerLog[name] for name in players_test)

In [28]:
target_df_train = df_train[target]
target_df_val = df_val[target]
target_df_test = df_test[target]

In [29]:
target_df_train.shape

(76923,)

In [30]:
def convert_df(df, window_size=5):
    data = df.to_numpy()
    X, y = [], []

    for i in range(len(data) - window_size):
        X.append([[a] for a in data[i:i + 5]])
        y.append(data[i+5])

    return np.array(X), np.array(y)

In [31]:
WINDOW_SIZE = 5

X_train, y_train = convert_df(target_df_train, WINDOW_SIZE)
X_val, y_val = convert_df(target_df_val, WINDOW_SIZE)
X_test, y_test = convert_df(target_df_test, WINDOW_SIZE)

In [32]:
X_train.shape, y_train.shape

((76918, 5, 1), (76918,))

### LSTM Model Training

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [34]:
model1 = Sequential()
model1.add(InputLayer((5, 1)))
model1.add(LSTM(10))
model1.add(Dense(8, 'relu'))
model1.add(Dense(1, 'linear'))

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10)                480       
                                                                 
 dense (Dense)               (None, 8)                 88        
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 577
Trainable params: 577
Non-trainable params: 0
_________________________________________________________________


In [58]:
cp = ModelCheckpoint('model1/', save_best_only=True)
model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.01), metrics=[RootMeanSquaredError()])

In [59]:
model1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, callbacks=[cp])

Epoch 1/10
2727/2728 [============================>.] - ETA: 0s - loss: 46.5014 - root_mean_squared_error: 6.8192

INFO:tensorflow:Assets written to: model1\assets


INFO:tensorflow:Assets written to: model1\assets


2728/2728 [==============================] - 37s 13ms/step - loss: 46.4992 - root_mean_squared_error: 6.8190 - val_loss: 42.5173 - val_root_mean_squared_error: 6.5205
Epoch 2/10
2727/2728 [============================>.] - ETA: 0s - loss: 43.6620 - root_mean_squared_error: 6.6077

INFO:tensorflow:Assets written to: model1\assets


INFO:tensorflow:Assets written to: model1\assets


2728/2728 [==============================] - 30s 11ms/step - loss: 43.6627 - root_mean_squared_error: 6.6078 - val_loss: 41.8983 - val_root_mean_squared_error: 6.4729
Epoch 3/10
2723/2728 [============================>.] - ETA: 0s - loss: 43.5489 - root_mean_squared_error: 6.5992

INFO:tensorflow:Assets written to: model1\assets


INFO:tensorflow:Assets written to: model1\assets


2728/2728 [==============================] - 32s 12ms/step - loss: 43.5451 - root_mean_squared_error: 6.5989 - val_loss: 41.6990 - val_root_mean_squared_error: 6.4575
Epoch 4/10
2728/2728 [==============================] - 42s 15ms/step - loss: 43.4906 - root_mean_squared_error: 6.5947 - val_loss: 42.5990 - val_root_mean_squared_error: 6.5268
Epoch 5/10
2728/2728 [==============================] - 38s 14ms/step - loss: 43.4616 - root_mean_squared_error: 6.5925 - val_loss: 41.8700 - val_root_mean_squared_error: 6.4707
Epoch 6/10
2728/2728 [==============================] - 39s 14ms/step - loss: 43.3992 - root_mean_squared_error: 6.5878 - val_loss: 41.8449 - val_root_mean_squared_error: 6.4688
Epoch 7/10
2728/2728 [==============================] - 36s 13ms/step - loss: 43.3823 - root_mean_squared_error: 6.5865 - val_loss: 41.8824 - val_root_mean_squared_error: 6.4717
Epoch 8/10
2728/2728 [==============================] - 33s 12ms/step - loss: 43.3805 - root_mean_squared_error: 6.5864 -